In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('https://raw.githubusercontent.com/masumhasan/Movie-Recommender-AI/main/tmdb_5000_movies.csv')
credits = pd.read_csv('https://raw.githubusercontent.com/masumhasan/Movie-Recommender-AI/main/tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)['cast'].values

In [ ]:
credits.head(1)['crew'].values
# show 1 crew values

Marge 2 dataset

In [ ]:
#dataset marge code

#movies.merge(credits,on='title')
#check size
# movies.merge(credits,on='title').shape
#assign new df in movies
movies = movies.merge(credits,on='title')

In [ ]:
#check size
movies.shape
#credits.shape

(4809, 23)

DATA TRIMMING DROP UNWANTED COLUMNS OF MOVIES DATAFRAME


In [ ]:
#df info
movies.info()

In [ ]:
#movies_id
#title
#genres
#keywords
#overview
#cast
#crew

In [ ]:
#storing only prefered columns back into movies df
movies = movies[['movie_id','title','genres','overview','keywords','cast','crew']]

In [ ]:
#df info
movies.info()

In [ ]:
movies.head(1)

Data Preprocessing


In [ ]:
#check for missing data
movies.isnull().sum()

In [ ]:
#Drop missing datas
movies.dropna(inplace=True)

In [ ]:
#Check for duplicate data
movies.duplicated().sum()

In [ ]:
#Check data
movies.iloc[0].genres

Preprocessing



In [ ]:
#GOAL
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
#['Action','Adventure','Fantacy']

In [ ]:
#Convert string of list to list 
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
#helper function
def converttoL(obj):
    L= []
    for i in ast.literal_eval(obj): #using ast literal_eval function from string = list
        L.append(i['name'])
    return L    

In [ ]:
#converting genres to a list from string
movies['genres'] = movies['genres'].apply(converttoL) 

In [ ]:
movies.head(1)

In [ ]:
#converting keywords to a list from string
movies['keywords'] = movies['keywords'].apply(converttoL) 

In [ ]:
movies.head(1)

In [ ]:
#helper function
def convertcast(obj):
    L= []
    counter = 0
    for i in ast.literal_eval(obj): #using ast literal_eval function from string = list
        if counter !=4: #looping for 4 cast only
            L.append(i['name'])
            counter +=1
        else:
             break    
    return L    

In [ ]:
#converting casts to a list from string
movies['cast'] = movies['cast'].apply(convertcast) 

In [ ]:
movies['cast'][0]

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Stephen Lang']

In [ ]:
movies.head()

In [ ]:
#Helper function
def fDirector(obj):
    L= []
    for i in ast.literal_eval(obj): #using ast literal_eval function from string = list
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L    

In [ ]:
#converting casts (Director) to a list from string
movies['crew'] = movies['crew'].apply(fDirector)

In [ ]:
movies.head()

overview

In [ ]:
movies['overview'][0]
#its a string

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [ ]:
# spliting the overview data into words with lambda split function
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(4)

Removing Spaces from data

In [ ]:
#Science Fiction to ScienceFiction
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head(2)

Adding all data to tags

In [ ]:
movies['tags']= movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

NEW DATAFRAME

In [ ]:
nFrame = movies[['movie_id','title','tags']]

In [ ]:
nFrame

Converting Tags to string

In [ ]:
#Converting Tags to string
nFrame['tags'] = nFrame['tags'].apply(lambda x:" ".join(x))

In [ ]:
nFrame.head()

In [ ]:
nFrame['tags'][0] #overview of 0th data

In [ ]:
#convert to lowercase
nFrame['tags'] = nFrame['tags'].apply(lambda x:x.lower())

In [ ]:
nFrame.head()

Preprocessing Complete

Converting data to vector

In [ ]:
#remove stopwords like a an the and or 
#making vector for 5000 words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
#converting nFrame ['tags'] to vector
#cv.fit_transform(nFrame['tags']).toarray().shape
vectors = cv.fit_transform(nFrame['tags']).toarray()

In [ ]:
cv.get_feature_names_out()
#len(cv.get_feature_names_out())

In [ ]:
import nltk
#import nltk NLP library to stem words

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
#sample
ps.stem('dancing')

'danc'

In [ ]:
#helper function
def wordRoot(text):
    w=[]
    for i in text.split():
        w.append(ps.stem(i))
    return " ".join(w)


In [ ]:
#test
wordRoot('hello im loving this system')

In [ ]:
nFrame['tags'] = nFrame['tags'].apply(wordRoot)

Calculating Movie Vector cosine distaces

In [ ]:
#Calculating Movie Vector cosine distaces
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#cosine_similarity(vectors).shape
match = cosine_similarity(vectors)

In [ ]:
#Similarity Score
match[0]

array([1.        , 0.08964215, 0.05884899, ..., 0.02548236, 0.02817181,
       0.        ])

In [ ]:
#Recomendation for movie index 0
sorted(list(enumerate(match[0])), reverse=True, key=lambda x:x[1])[1:6]

In [ ]:
#Fetch Index of movie
nFrame[nFrame['title'] == 'The Social Network'].index[0]

1164

In [ ]:
#fetch title from index
nFrame.iloc[1216].title


'Aliens vs Predator: Requiem'

In [ ]:
def recommend(movie):
    movie_index = nFrame[nFrame['title'] == movie].index[0]
    distances = match[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(nFrame.iloc[i[0]].title)
        #print(i[0])
    
 

In [ ]:
recommend('Jurassic Park')

Jurassic World
The Lost World: Jurassic Park
Jurassic Park III
The Nut Job
Walking With Dinosaurs
